### Dependencies

All the dependencies for this notebook to run on colab are included in the `requirements_colab.txt` file included in this folder. To run the model on GPU you should go to edit/notebook settings and select GPU

In [ ]:
!pip install -r 'https://raw.githubusercontent.com/radiantearth/Nasa_harvest_field_boundary_competition/main/requirements_colab.txt'

In [ ]:
!pip install tensorflow_addons

In [ ]:
# Importing the needed libraries
import getpass
import glob
import keras
import os
import pickle
import random
import shutil
from radiant_mlhub import Dataset

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio as rio

import tensorflow as tf
import segmentation_models as sm

from pathlib import Path
from radiant_mlhub import Dataset
from random import choice
from segmentation_models import Unet
from scipy.ndimage import gaussian_filter

import torch


from keras import backend as K
from keras.layers import *
from keras.models import *
from keras.models import load_model
from keras.optimizers import *
from keras.preprocessing import image

from tensorflow.keras.layers import *
from tensorflow.keras.losses import *

from keras.callbacks import ModelCheckpoint

from tensorflow.python.keras.backend import resize_images
import tensorflow_addons as tfa

import cv2
import gc


from typing import List, Any, Callable, Tuple

In [ ]:
seed = 2023

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed=2023)

In [ ]:
dataset_id = 'nasa_rwanda_field_boundary_competition'
assets = ['labels']

In [ ]:
#Append your MLHUB_API_KEY after this cell is executed to download dataset
os.environ['MLHUB_API_KEY'] = getpass.getpass(prompt="MLHub API Key: ")
dataset = Dataset.fetch(dataset_id)

In [ ]:
dataset.download(output_dir = dataset_id, if_exists='overwrite')

In [ ]:
archives = ['source_train', 'source_test', 'labels_train']

In [ ]:
for archive in archives:
    full_path = f"{dataset_id}/{dataset_id}_{archive}.tar.gz"
    shutil.unpack_archive(full_path, dataset_id)

In [ ]:
#image snapshot dimensions
IMG_WIDTH = 256 
IMG_HEIGHT = 256 
IMG_CHANNELS = 4 #we have the rgba bands

We have two sets of data: the train and test dataset, each having a list of file ids belonging to them.
For model development purposes, we will use the training set(`train_tiles`) and use the test set for model prediction/evaluation.

In [ ]:
train_source_items = f"{dataset_id}/{dataset_id}_source_train"
train_label_items = f"{dataset_id}/{dataset_id}_labels_train"

In [ ]:
next(os.walk(train_source_items))[1][0]

'nasa_rwanda_field_boundary_competition_source_train_50_2021_10'

In [ ]:
def clean_string(s: str) -> str:
    """
    extract the tile id and timestamp from a source image folder
    e.g extract 'ID_YYYY_MM' from 'nasa_rwanda_field_boundary_competition_source_train_ID_YYYY_MM'
    """
    s = s.replace(f"{dataset_id}_source_", '').split('_')[1:]
    return '_'.join(s)

In [ ]:
train_tiles = [clean_string(s) for s in next(os.walk(train_source_items))[1]]

In [ ]:
#loading the 4 bands of the image
tile = random.choice(train_tiles)
print(tile)
bd1 = rio.open(f"{train_source_items}/{dataset_id}_source_train_{tile}/B01.tif")
bd1_array = bd1.read(1)
bd2 = rio.open(f"{train_source_items}/{dataset_id}_source_train_{tile}/B02.tif")
bd2_array = bd2.read(1)
bd3 = rio.open(f"{train_source_items}/{dataset_id}_source_train_{tile}/B03.tif")
bd3_array = bd3.read(1)
bd4 = rio.open(f"{train_source_items}/{dataset_id}_source_train_{tile}/B04.tif")
bd4_array = bd4.read(1)

field = np.dstack((bd4_array, bd3_array, bd2_array, bd1_array))

field = np.sqrt(field)

#data standardization
for c in range(field.shape[2]):
    mean = field[:, :, c].mean()
    std = field[:, :, c].std()
    field[:, :, c] = (field[:, :, c] - mean) / std


mask  = rio.open(Path.cwd() / f"{train_label_items}/{dataset_id}_labels_train_{tile.split('_')[0]}/raster_labels.tif").read(1)

### Data Augmentation

In [ ]:
#https://github.com/radix-ai/agoro-field-boundary-detector/tree/master/src/agoro_field_boundary_detector
def t_linear(
    field: np.ndarray,
    mask: np.ndarray,
    _: int = 0,
) -> Tuple[np.ndarray, np.ndarray]:
    """Apply a linear (i.e. no) transformation and save."""
    return field, mask

def t_quartile(
    field: np.ndarray,
    mask: np.ndarray,
    idx: int,
) -> Tuple[np.ndarray, np.ndarray]:
    """Divide the information into four quarters."""
    assert idx in range(0, 3 + 1)
    x, y = [(0, 0), (0, 1), (1, 0), (1, 1)][idx]
    width, height = mask.shape  # 2d array

    # Slice and recover shape
    field_slice = field[
        (width // 2) * x : (width // 2) * (x + 1), (height // 2) * y : (height // 2) * (y + 1)
    ]
    field_slice = field_slice.repeat(2, axis=0).repeat(2, axis=1)
    mask_slice = mask[
        (width // 2) * x : (width // 2) * (x + 1), (height // 2) * y : (height // 2) * (y + 1)
    ]
    mask_slice = mask_slice.repeat(2, axis=0).repeat(2, axis=1)

    # Normalise masking values
    values = sorted(set(np.unique(mask_slice)) - {0})
    for idx, v in enumerate(values):
        mask_slice[mask_slice == v] = idx + 1
    return field_slice, mask_slice
    
def t_rotation(
    field: np.ndarray,
    mask: np.ndarray,
    rot: int,
) -> Tuple[np.ndarray, np.ndarray]:
    """Rotate the data."""
    assert rot in range(0, 3 + 1)
    for _ in range(rot):
        field = np.rot90(field)
        mask = np.rot90(mask)
    return field, mask

def t_flip(
    field: np.ndarray,
    mask: np.ndarray,
    idx: int,
) -> Tuple[np.ndarray, np.ndarray]:
    """Flip the data."""
    assert idx in range(0, 2 + 1)
    if idx == 0:  # Diagonal
        field = np.rot90(np.fliplr(field))
        mask = np.rot90(np.fliplr(mask))
    if idx == 1:  # Horizontal
        field = np.flip(field, axis=0)
        mask = np.flip(mask, axis=0)
    if idx == 2:  # Vertical
        field = np.flip(field, axis=1)
        mask = np.flip(mask, axis=1)
    return field, mask

def t_blur(
    field: np.ndarray,
    mask: np.ndarray,
    sigma: int,
) -> Tuple[np.ndarray, np.ndarray]:
    """Blur the image by applying a Gaussian filter."""
    assert 0 <= sigma <= 10
    sigma_f = 1.0 + (sigma / 10)
    field = np.copy(field)
    for i in range(3):
        field[:, :, i] = gaussian_filter(field[:, :, i], sigma=sigma_f)
    return field, mask

In [ ]:
def show_image(field:np.ndarray, mask:np.ndarray): 
    """Show the field and corresponding mask."""
    fig = plt.figure(figsize=(8,6))
    ax1 = fig.add_subplot(121)  # left side
    ax2 = fig.add_subplot(122)  # right side
    ax1.imshow(field[:,:,0:3])  # rgb band
    plt.gray()
    ax2.imshow(mask)
    plt.tight_layout()
    plt.show()

In [ ]:
show_image(field,mask)

In [ ]:
f,m = t_flip(field, mask, idx=0) #flipping
show_image(f,m)

In [ ]:
f,m = t_blur(field, mask, sigma=5) #blur
show_image(f,m)

In [ ]:
def generate(
    field: np.ndarray,
    mask: np.ndarray,
    write_folder: Path,
    prefix: str = "",
) -> None:
    """
    Generate data augmentations of the provided field and corresponding mask which includes:
     - Linear (no) transformation
     - Rotation
     - Horizontal or vertical flip
     - Gaussian filter (blur)
    :param field: Input array of the field to augment
    :param mask: Input array of the corresponding mask to augment
    :param write_folder: Folder (path) to write the results (augmentations) to
    :param prefix: Field-specific prefix used when writing the augmentation results
    """
    # Generate transformations
    f, m = [0,1,2,3, 4], [0,1,2,3, 4] #dummy data. will be replaced
    f[0],m[0] = t_linear(field, mask) #no augmentation
    f[1],m[1] = t_rotation(field, mask, rot=1) #rotation
    f[2],m[2] = t_flip(field, mask, idx=0) #flipping
    f[3],m[3] = t_blur(field, mask, sigma=5) #blur
    f[4],m[4] = t_quartile(field, mask, idx=1) #quartile    
    
    for i in range(len(f)):        
    

        with open(write_folder +'/'+ f"fields/{str(prefix).zfill(2)}_{i}.pkl", 'wb') as handle:
            pickle.dump(f[i], handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open(write_folder +'/'+ f"masks/{str(prefix).zfill(2)}_{i}.pkl", 'wb') as handle:
            pickle.dump(m[i], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def main(
    field: List[np.ndarray],
    mask: List[np.ndarray],
    prefix: List[str],
    write_folder: Path,
) -> None:
    """
    Generate and save data augmentations for all the fields and corresponding masks with the following:
     - Linear (no) transformation
     - Rotation
     - Horizontal or vertical flip
     - Gaussian filter (blur)
     - Gamma filter (brightness)
    :param fields: Fields to augment
    :param masks: Corresponding masks to augment
    :param prefixes: Field-specific prefixes corresponding each field
    :param write_folder: Path to write the results (augmentations) to
    """
    generate(
        field=field,
        mask=mask,
        prefix=prefix,
        write_folder=write_folder,
    )

In [ ]:
#apply augmentation effects to training set
for tile in train_tiles:
    bd1 = rio.open(f"{train_source_items}/{dataset_id}_source_train_{tile}/B01.tif")
    bd1_array = bd1.read(1)
    bd2 = rio.open(f"{train_source_items}/{dataset_id}_source_train_{tile}/B02.tif")
    bd2_array = bd2.read(1)
    bd3 = rio.open(f"{train_source_items}/{dataset_id}_source_train_{tile}/B03.tif")
    bd3_array = bd3.read(1)
    bd4 = rio.open(f"{train_source_items}/{dataset_id}_source_train_{tile}/B04.tif")
    bd4_array = bd4.read(1)

    field = np.dstack((bd4_array, bd3_array, bd2_array, bd1_array))

    field = np.sqrt(field)

    #data standardization
    for c in range(field.shape[2]):
        mean = field[:, :, c].mean()
        std = field[:, :, c].std()
        field[:, :, c] = (field[:, :, c] - mean) / std


    ids_list  = tile.split('_') # XX_YYYY_MM where XX is the training file id and YYYY_MM is the timestamp
    tile_id   = ids_list[0]
    timestamp = f"{ids_list[1]}_{ids_list[2]}"

    mask  = rio.open(Path.cwd() / f"{train_label_items}/{dataset_id}_labels_train_{tile_id}/raster_labels.tif").read(1) 

    #create a folder for the augmented images
    if not os.path.isdir(f"./train_data/{timestamp}"):
        os.makedirs(f"./train_data/{timestamp}")
    if not os.path.isdir(f"./train_data/{timestamp}/fields"):
        os.makedirs(f"./train_data/{timestamp}/fields")
    if not os.path.isdir(f"./train_data/{timestamp}/masks"):
        os.makedirs(f"./train_data/{timestamp}/masks")

    main( #applying augmentation effects
        field  = field,
        mask   = mask,
        prefix = tile_id,
        write_folder = f"./train_data/{timestamp}",
    ) 

In [ ]:
timestamps = next(os.walk(f"./train_data"))[1] #Get all timestamps
augmented_files = next(os.walk(f"./train_data/{timestamps[0]}/fields"))[2] #Get all augmented tile ids. can just use one timestamp
X = np.empty((len(augmented_files), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS*len(timestamps)), dtype=np.float32) #time-series image
y = np.empty((len(augmented_files), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.uint8) #mask for each scene
i = 0
for file in augmented_files:
    idx = 0
    augmented_id = file.split('.pkl')[0] #id without .pkl extension
    temporal_fields = []
    for timestamp in timestamps:
        with open(f"./train_data/{timestamp}/fields/{augmented_id}.pkl", 'rb') as field:
            field = pickle.load(field) 
            
        X[i][:,:,idx:idx+IMG_CHANNELS] = field
        idx += IMG_CHANNELS
    with open(f"./train_data/{timestamp}/masks/{augmented_id}.pkl", 'rb') as mask:
        mask = pickle.load(mask)
    y[i] = mask.reshape(IMG_HEIGHT, IMG_WIDTH, 1)
    i+=1

In [ ]:
X.shape

In [ ]:
random.randint(0, len(augmented_files)) #sanity check
random_image = random.randint(0, len(augmented_files)-1)
show_image(X[random_image][:,:,0:3], y[random_image].reshape(IMG_WIDTH, IMG_HEIGHT))

# Scheduler

In [ ]:
class SGDRScheduler(tf.keras.callbacks.Callback):
    '''Cosine annealing learning rate scheduler with periodic restarts.
    # Usage
        ```python
            schedule = SGDRScheduler(min_lr=1e-5,
                                     max_lr=1e-2,
                                     steps_per_epoch=np.ceil(epoch_size/batch_size),
                                     lr_decay=0.9,
                                     cycle_length=5,
                                     mult_factor=1.5)
            model.fit(X_train, Y_train, epochs=100, callbacks=[schedule])
        ```
    # Arguments
        min_lr: The lower bound of the learning rate range for the experiment.
        max_lr: The upper bound of the learning rate range for the experiment.
        steps_per_epoch: Number of mini-batches in the dataset. Calculated as `np.ceil(epoch_size/batch_size)`. 
        lr_decay: Reduce the max_lr after the completion of each cycle.
                  Ex. To reduce the max_lr by 20% after each cycle, set this value to 0.8.
        cycle_length: Initial number of epochs in a cycle.
        mult_factor: Scale epochs_to_restart after each full cycle completion.
    # References
        Blog post: jeremyjordan.me/nn-learning-rate
        Original paper: http://arxiv.org/abs/1608.03983
    '''
    def __init__(self,
                 min_lr,
                 max_lr,
                 steps_per_epoch,
                 lr_decay=1,
                 cycle_length=10,
                 mult_factor=2):

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.lr_decay = lr_decay

        self.batch_since_restart = 0
        self.next_restart = cycle_length

        self.steps_per_epoch = steps_per_epoch

        self.cycle_length = cycle_length
        self.mult_factor = mult_factor

        self.history = {}

    def clr(self):
        '''Calculate the learning rate.'''
        fraction_to_restart = self.batch_since_restart / (self.steps_per_epoch * self.cycle_length)
        lr = self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(fraction_to_restart * np.pi))
        return lr

    def on_train_begin(self, logs={}):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.max_lr)

    def on_batch_end(self, batch, logs={}):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        self.batch_since_restart += 1
        K.set_value(self.model.optimizer.lr, self.clr())

    def on_epoch_end(self, epoch, logs={}):
        '''Check for end of current cycle, apply restarts when necessary.'''
        if epoch + 1 == self.next_restart:
            self.batch_since_restart = 0
            self.cycle_length = np.ceil(self.cycle_length * self.mult_factor)
            self.next_restart += self.cycle_length
            self.max_lr *= self.lr_decay
            self.best_weights = self.model.get_weights()

    def on_train_end(self, logs={}):
        '''Set weights to the values from the end of the most recent cycle for best performance.'''
        self.model.set_weights(self.best_weights)

# Training Hyperparameters

In [ ]:
BACKBONE = 'efficientnetb7'


num_channels = 24
input_shape = (256,256,num_channels)


BATCH_SIZE = 4 

LR = 1e-4

N_EPOCHS = 50

MIN_LR = 1e-6

MAX_LR = 1e-2

MULTI_FACTOR = 1.5

DECAY_RATE = 0.85

WEIGHT_DECAY = 1e-5

In [ ]:
#https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
def f1(y_true, y_pred):
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall_   = recall(y_true, y_pred)
    return 2*((precision*recall_)/(precision+recall_+K.epsilon()))

In [ ]:
# Combo Loss - Dice + Binary crossentropy

def dice_coef(y_true, y_pred, smooth=1.0):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def binary_crossentropy(y, p):
    return K.mean(K.binary_crossentropy(y, p))


def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)


def dice_coef_loss_bce(y_true, y_pred, dice=0.5, bce=0.5):
    return binary_crossentropy(y_true, y_pred) * bce + dice_coef_loss(y_true, y_pred) * dice

def bce_dice_loss(y, p):
    return dice_coef_loss_bce(y, p, dice=0.9, bce=0.1)

In [ ]:
sm.set_framework('tf.keras')
sm.framework()

def get_model():
    model = None 
    model_unet = Unet(BACKBONE, encoder_weights='imagenet')
    new_model = keras.models.Sequential()
    new_model.add(Conv2D(3, (1,1), padding='same', activation='relu', input_shape=input_shape))
    new_model.add(model_unet)
    model = new_model 

    return model

In [ ]:
tf.config.experimental_run_functions_eagerly(True)

In [ ]:
# reference : https://www.kaggle.com/code/giselama/5-1-multilabel-semantic-segmentation

def stratified_split(x, y, val_size = 0.2, kFold=5):
    ind = np.arange(len(y))
    np.random.shuffle(ind)
    coverage = []
    for i in range(0, len(y)):
        coverage.append(np.sum(y[ind[i]]))

    hist, bin_edges = np.histogram(coverage)
    bin_edges[len(bin_edges)-1] = bin_edges[len(bin_edges)-1] + 1
    cindex = np.digitize(coverage,bin_edges)

    val_size = val_size
    for ii in range(kFold): #5-fold learning
        k = ii
        train_idxs = []
        val_idxs = []
        for i in range(0,10):
            index_temp = ind[cindex==i+1]
            list_temp = index_temp.T.tolist()
            val_samples = round(len(index_temp)*val_size)
            if (k == 0):
                val_idxs = val_idxs + list_temp[:val_samples]
                train_idxs = train_idxs + list_temp[val_samples:]
            elif (k == 4):
                val_idxs = val_idxs + list_temp[4*val_samples:]
                train_idxs = train_idxs + list_temp[:4*val_samples]
            else:
                val_idxs = val_idxs + list_temp[k*val_samples:(k+1)*val_samples]
                train_idxs = train_idxs + list_temp[:k*val_samples] + list_temp[(k+1)*val_samples:]

        val_idxs = ind[val_idxs]
        train_idxs = ind[train_idxs]

    return x[train_idxs],y[train_idxs],x[val_idxs],y[val_idxs]


# Creating Train and Validation splits

In [ ]:
X_train, y_train,X_valid, y_valid  = stratified_split(X, y, val_size = 0.2, kFold=5)

In [ ]:
X_train.shape, X_valid.shape

In [ ]:
from scipy import ndimage

In [ ]:
# Load all train tiles

bands_arr = np.zeros((len(train_tiles), IMG_HEIGHT, IMG_WIDTH, 4), dtype = np.float32)

for tid, tile in enumerate(train_tiles):

    bd1 = rio.open(f"{train_source_items}/{dataset_id}_source_train_{tile}/B01.tif")
    bd1_array = bd1.read(1)
    bd2 = rio.open(f"{train_source_items}/{dataset_id}_source_train_{tile}/B02.tif")
    bd2_array = bd2.read(1)
    bd3 = rio.open(f"{train_source_items}/{dataset_id}_source_train_{tile}/B03.tif")
    bd3_array = bd3.read(1)
    bd4 = rio.open(f"{train_source_items}/{dataset_id}_source_train_{tile}/B04.tif")
    bd4_array = bd4.read(1)

    bands_arr[tid] = np.dstack((bd4_array, bd3_array, bd2_array, bd1_array))
    

bands_arr = np.sqrt(bands_arr)

#data standardization
for c in range(bands_arr.shape[-1]):
    mean = bands_arr[:, :, :, c].mean()
    std = bands_arr[:, :, :, c].std()
    bands_arr[:, :, :, c] = (bands_arr[:, :, :, c] - mean) / std



tiles = bands_arr.copy()

del bands_arr
_ = gc.collect()

# Mixup Data Augmentation

In [ ]:
# Adapted from : https://github.com/radiantearth/CropDetectionDL/blob/main/dataset.py

def augment(img, mask):       

    p = np.random.random(3)
    ang = np.random.uniform(-10, 10)

    #mixup training image with a random crop from the tiles
    if (p[0] > 0.5):
        start_x = np.random.randint(0, bands_arr.shape[1] - 32)
        start_y = np.random.randint(0, bands_arr.shape[2] - 32)
        t = np.random.randint(0, 342)
        rnd_crop = tiles[t] # randomly sample a tile
        d = 0.85

        img = img.reshape(6, IMG_WIDTH, IMG_HEIGHT, 4)
        img = img * d + rnd_crop * (1 - d)

        img = img.reshape(IMG_WIDTH, IMG_HEIGHT, 24)


    #apply flipping and rotation augmentation

    if p[1] > 0.5:
        mask[0] = np.flipud(mask[0])
    if p[2] > 0.5:
        mask = ndimage.rotate(mask, ang, reshape = False)        
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if p[1] > 0.5:
                img[i,j] = np.flipud(img[i,j])

    return img, mask

In [ ]:
xtrain_aug2, ytrain2 =[],[]

for i in range(len(X_train)):
    
    xx, yy = augment(X_train[i], y_train[i])
    
    xtrain_aug2.append(xx)
    ytrain2.append(yy)

In [ ]:
X_augmented =   np.array(xtrain_aug2, dtype = np.float32)
Y_augmented = np.array(ytrain2,  dtype = np.float32)

In [ ]:
X_augmented.shape, Y_augmented.shape

In [ ]:
del  xtrain_aug2,ytrain2
_ = gc.collect()

In [ ]:
X_train = np.vstack((X_train, X_augmented))
y_train = np.vstack((y_train, Y_augmented))

In [ ]:
X_train.shape, y_train.shape

# Train

In [ ]:
seed_everything(seed=2023)

K.clear_session()

y_valid = tf.cast(y_valid, tf.float32)
y_train = tf.cast(y_train, tf.float32)   

mc = ModelCheckpoint(f'./best_model.h5', monitor='val_f1', verbose=0, save_best_only=True,
                                    save_weights_only=True, mode='max')

sched = SGDRScheduler(min_lr= MIN_LR,
                                  max_lr=MAX_LR,
                                  steps_per_epoch=np.ceil(N_EPOCHS/BATCH_SIZE),
                                  lr_decay=DECAY_RATE,
                                  mult_factor=MULTI_FACTOR)

cbs = [sched, mc]


model = get_model()

model.compile(loss= bce_dice_loss,
              optimizer=tfa.optimizers.AdamW(learning_rate=LR, weight_decay=WEIGHT_DECAY),
              metrics=[f1, recall])

history = model.fit(X_train,y_train,
              validation_data=(X_valid, y_valid),batch_size = BATCH_SIZE,
              steps_per_epoch = len(X_train)//BATCH_SIZE,
              validation_steps = np.ceil(len(X_valid)/BATCH_SIZE), epochs=N_EPOCHS,
              callbacks=cbs)

del model, X_train, X_valid, y_train, y_valid
_ =gc.collect()


loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label=f'Fold {fold} Training loss')
plt.plot(epochs, val_loss, 'r', label='Fold {fold} Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


recall_score = history.history['recall']
val_recall_score = history.history['val_recall']
epochs = range(1, len(recall_score) + 1)
plt.plot(epochs, recall_score, 'y', label=f'Fold {fold} Training recall')
plt.plot(epochs, val_recall_score, 'r', label=f'Fold {fold} Validation recall')
plt.title('Training and validation Recall')
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.legend()
plt.show()

f1score = history.history['f1']
val_f1score = history.history['val_f1']
epochs = range(1, len(f1score) + 1)
plt.plot(epochs, f1score, 'y', label=f'Fold {fold} Training f1')
plt.plot(epochs, val_f1score, 'r', label=f'Fold {fold} Validation f1')
plt.title('Training and validation f1')
plt.xlabel('Epochs')
plt.ylabel('f1 ')
plt.legend()
plt.show()


del history
_ = gc.collect()

# Loading test chips to run predictions



In [ ]:
test_source_items = f"{dataset_id}/{dataset_id}_source_test"
test_tiles = [clean_string(s) for s in next(os.walk(test_source_items))[1]]

In [ ]:
test_tile_ids = set()
for tile in test_tiles:
    test_tile_ids.add(tile.split('_')[0])

In [ ]:
X_test = np.empty((len(test_tile_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS*len(timestamps)), dtype=np.float32)
i = 0
loaded_tiles = []
for tile_id in test_tile_ids:
    idx = 0
    for timestamp in timestamps:
        bd1 = rio.open(f"{test_source_items}/{dataset_id}_source_test_{tile_id}_{timestamp}/B01.tif")
        bd1_array = bd1.read(1)
        bd2 = rio.open(f"{test_source_items}/{dataset_id}_source_test_{tile_id}_{timestamp}/B02.tif")
        bd2_array = bd2.read(1)
        bd3 = rio.open(f"{test_source_items}/{dataset_id}_source_test_{tile_id}_{timestamp}/B03.tif")
        bd3_array = bd3.read(1)
        bd4 = rio.open(f"{test_source_items}/{dataset_id}_source_test_{tile_id}_{timestamp}/B04.tif")
        bd4_array = bd4.read(1)

        field = np.dstack((bd4_array, bd3_array, bd2_array, bd1_array))


        field = np.sqrt(field)

        #data standardization
        for c in range(field.shape[2]):
            mean = field[:, :, c].mean()
            std = field[:, :, c].std()
            field[:, :, c] = (field[:, :, c] - mean) / std
            
        X_test[i][:,:,idx:idx+IMG_CHANNELS] = field
        idx+=IMG_CHANNELS
    loaded_tiles.append(str(tile_id).zfill(2)) #track order test tiles are loaded into X to make sure tile id matches 
    i+=1

In [ ]:
# load best model to run predictions

model.load_weights("./best_model.h5")

preds = model.predict(X_test, verbose=1, batch_size=batch_size)

In [ ]:
preds.shape

In [ ]:
preds.min(), preds.max()

In [ ]:
preds = np.clip(preds, 0.0001, 0.9999)

In [ ]:
predictions_dictionary = {}
for i in range(len(test_tile_ids)):
    model_pred = preds[i]
    model_pred = (model_pred >= 0.5).astype(np.uint8)
    model_pred = model_pred.reshape(IMG_HEIGHT, IMG_WIDTH)
    predictions_dictionary.update([(str(loaded_tiles[i]), pd.DataFrame(model_pred))])

In [ ]:
dfs = []
for key, value in predictions_dictionary.items():
    ftd = value.unstack().reset_index().rename(columns={'level_0': 'row', 'level_1': 'column', 0: 'label'})
    ftd['tile_row_column'] = f'Tile{key}_' + ftd['row'].astype(str) + '_' + ftd['column'].astype(str)
    ftd = ftd[['tile_row_column', 'label']]
    dfs.append(ftd)

sub = pd.concat(dfs)
sub

In [ ]:
sub.label.value_counts()

In [ ]:
sub.to_csv("sub_b7.csv", index = False)